# Census tract to Chicago community area aggregation by poverty status

<b>Methodology:</b> Similar to race and ethnicity.

In [1]:
# configs
import pandas as pd

Downloaded 2023 5-year ACS data for B17001, Poverty Status in the last 12 months, by sex by age, census tract for Cook County

In [3]:
# load census data
acs = pd.read_csv('../census-data/ACSDT5Y2023.B17001_2025-07-11T165950/ACSDT5Y2023.B17001-Data.csv', skiprows=[1],
                  header=0,
                  usecols=['GEO_ID',
                           'B17001_001E', # total
                           'B17001_002E',]) # population assessed poverty

In [4]:
acs.head(2)

,GEO_ID,B17001_001E,B17001_002E
0,1400000US17031010100,3627,508
1,1400000US17031010201,7578,1892


In [5]:
# rename cols
acs.columns = ['geoid', 'total', 'population_assessed_poverty']

In [6]:
# clean id to match crosswalk tract geoid
acs['geoid'] = acs['geoid'].astype(str)
acs['geoid_clean'] = acs['geoid'].str.replace('1400000US', '')
acs['geoid_clean'] = acs['geoid_clean'].astype(int)

In [7]:
# add pcts 
acs['pct_poverty'] = acs['population_assessed_poverty']/acs['total']

In [8]:
# inspect
acs.sort_values('pct_poverty', ascending=False).head()

,geoid,total,population_assessed_poverty,geoid_clean,pct_poverty
1272,1400000US17031836900,2237,1642,17031836900,0.734019
424,1400000US17031400800,3384,2418,17031400800,0.714539
1261,1400000US17031835600,539,376,17031835600,0.697588
336,1400000US17031260400,1573,1049,17031260400,0.666879
604,1400000US17031671100,892,578,17031671100,0.647982


In [9]:
# load cmap crosswalk
crosswalk = pd.read_csv('../census-data/Crosswalk_TR_to_CCA_2020 (1).csv')
crosswalk.head(2)

,TRACT,GEOID,CCA,TR_POP_RAT,TR_HH_RAT,TR_HU_RAT
0,17031010100,1,Rogers Park,1.0,1.0,1.0
1,17031010201,1,Rogers Park,1.0,1.0,1.0


In [10]:
# left merge crosswalk with census data so duplicate keys have same census info
merged = pd.merge(crosswalk, acs, left_on='TRACT', right_on='geoid_clean', how='left', indicator=True)

In [12]:
# multiply by population ratio
merged['est_total'] = merged['total'] * merged['TR_POP_RAT']

In [13]:
# multiply est total by poverty rate
merged['est_population_assessed_poverty'] = merged['pct_poverty'] * merged['est_total']

In [14]:
agg = merged.groupby('CCA')[['est_total',
                             'est_population_assessed_poverty',]].sum().reset_index()

agg

,CCA,est_total,est_population_assessed_poverty
0,Albany Park,45159.000000,6149.000000
1,Archer Heights,13942.000000,1255.000000
2,Armour Square,14222.256914,3522.512496
3,Ashburn,41564.000000,6641.000000
4,Auburn Gresham,44985.000000,11209.000000
...,...,...,...
72,West Lawn,32419.000000,5058.000000
73,West Pullman,24433.000000,5991.000000
74,West Ridge,76949.000000,13417.000000
75,West Town,85878.297759,7664.249145


In [15]:
# add pcts
agg['pct_est_population_assessed_poverty'] = agg['est_population_assessed_poverty']/agg['est_total']

In [17]:
agg.sort_values('pct_est_population_assessed_poverty')

,CCA,est_total,est_population_assessed_poverty,pct_est_population_assessed_poverty
24,Forest Glen,19456.000000,844.000000,0.043380
50,North Center,35288.000000,1651.000000,0.046786
45,Mount Greenwood,18122.000000,1019.000000,0.056230
9,Beverly,19377.000000,1136.000000,0.058626
35,Jefferson Park,26548.000000,1598.000000,0.060193
...,...,...,...,...
55,Oakland,6935.000000,2446.000000,0.352704
19,East Garfield Park,20297.000000,7914.000000,0.389910
68,Washington Park,13034.000000,5964.000000,0.457573
25,Fuller Park,2233.848057,1182.817432,0.529498


In [18]:
agg.to_csv('../processed/chicago_poverty_agg_community.csv', index=False)

In [19]:
# check
# should equal close to 2,707,648 
agg['est_total'].sum()

np.float64(2656605.1229302436)

In [20]:
# check
# should equal close to 447,143
agg['est_population_assessed_poverty'].sum()

np.float64(447032.61700288067)